****Generate music using Deeplearning****

In [33]:
#Reference: Case study from www.appliedai.com

Data sources:
[Abc](http://abc.sourceforge.net/NMD/)






In [ ]:
# Jigs (340 tunes)
# Hornpipes (65 tunes)
# Morris (31 tunes)
# Playford (15 tunes)
# Reels A-C (81 tunes)
# Reels D-G (84 tunes)
# Reels H-L (93 tunes)
# Reels M-Q (80 tunes)
# Reels R-T (92 tunes)
# Reels U-Z (34 tunes)
# Slip Jigs (11 tunes)
# Waltzes (52 tunes)
# Chrismas Carols and Songs (13 tunes)
# Ashover collection (46 tunes)

Download all the music file and store in "input1.txt"

In [1]:
import os
import json
import argparse
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

os.chdir("/content/drive/MyDrive/nlp_projects/music_generation/data")

In [4]:
file = open("input1.txt","r")
data=file.read()

In [9]:
data[:30]

'X: 1\nT:A and D\n% Nottingham Mu'

In [10]:
# character to index and vice-versa mappings
char_to_idx = { ch: i for (i, ch) in enumerate(sorted(list(set(data)))) }
print("Number of unique characters: " + str(len(char_to_idx)))

Number of unique characters: 93


In [11]:
idx_to_char = { i: ch for (ch, i) in char_to_idx.items() }
vocab_size = len(char_to_idx)

In [20]:
vocab_size

93

In [12]:
def build_model(batch_size, seq_len, vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(batch_size, seq_len)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=True, stateful=True))
        model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(vocab_size))) 
    model.add(Activation('softmax'))
    return model

In [13]:
model = build_model(16, 64,vocab_size )
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, 64, 512)             47616     
_________________________________________________________________
lstm (LSTM)                  (16, 64, 256)             787456    
_________________________________________________________________
dropout (Dropout)            (16, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_2 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_2 (Dropout)          (16, 64, 256)             0

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
T = np.asarray([char_to_idx[c] for c in data], dtype=np.int32) #convert complete text into numerical indices

BATCH_SIZE = 16
SEQ_LENGTH = 64
steps_per_epoch = (len(data) / BATCH_SIZE - 1) / SEQ_LENGTH 

In [19]:
T.shape[0]

452185

In [21]:
def read_batches(T, vocab_size):
    length = T.shape[0]; #452185
    batch_chars = int(length / BATCH_SIZE); 

    for start in range(0, batch_chars - SEQ_LENGTH, SEQ_LENGTH): 
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH)) # 16X64
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, vocab_size)) # 16X64X93
        for batch_idx in range(0, BATCH_SIZE): # (0,16)
            for i in range(0, SEQ_LENGTH): #(0,64)
                X[batch_idx, i] = T[batch_chars * batch_idx + start + i] # 
                Y[batch_idx, i, T[batch_chars * batch_idx + start + i + 1]] = 1
        yield X, Y

In [22]:
from tqdm import tqdm
epochs=100
for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
        
    losses, accs = [], []

    for i, (X, Y) in enumerate(read_batches(T, vocab_size)):
            
        #print(X.shape);

        loss, acc = model.train_on_batch(X, Y)
        #print('Batch {}: loss = {}, acc = {}'.format(i + 1, loss, acc))
        losses.append(loss)
        accs.append(acc)
    print("loss={} and accuracy={}".format(loss,acc))


Epoch 1/100
loss=1.6085069179534912 and accuracy=0.5537109375

Epoch 2/100
loss=1.2896875143051147 and accuracy=0.6083984375

Epoch 3/100
loss=1.1673935651779175 and accuracy=0.6201171875

Epoch 4/100
loss=1.0717928409576416 and accuracy=0.6591796875

Epoch 5/100
loss=1.0172667503356934 and accuracy=0.6787109375

Epoch 6/100
loss=0.9627405405044556 and accuracy=0.7001953125

Epoch 7/100
loss=0.9324321150779724 and accuracy=0.7109375

Epoch 8/100
loss=0.8986650705337524 and accuracy=0.716796875

Epoch 9/100
loss=0.8641549348831177 and accuracy=0.732421875

Epoch 10/100
loss=0.8449010848999023 and accuracy=0.7353515625

Epoch 11/100
loss=0.8234649300575256 and accuracy=0.73828125

Epoch 12/100
loss=0.813001811504364 and accuracy=0.7333984375

Epoch 13/100
loss=0.7735465168952942 and accuracy=0.75390625

Epoch 14/100
loss=0.7448613047599792 and accuracy=0.7548828125

Epoch 15/100
loss=0.7335025668144226 and accuracy=0.775390625

Epoch 16/100
loss=0.7338449358940125 and accuracy=0.7636718

In [23]:
model.save_weights("music_gen.h5")

In [24]:
def build_sample_model(vocab_size):
    model = Sequential()
    model.add(Embedding(vocab_size, 512, batch_input_shape=(1, 1)))
    for i in range(3):
        model.add(LSTM(256, return_sequences=(i != 2), stateful=True))
        model.add(Dropout(0.2))

    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    return model

In [25]:
vocab_size

93

In [26]:
model = build_sample_model(vocab_size)
model.load_weights("music_gen.h5")

In [27]:
sequence_index = [25] # any random number between 1-93
    
for _ in range(8000):
    batch = np.zeros((1, 1))
    batch[0, 0] = sequence_index[-1]
        
    predicted_probs = model.predict_on_batch(batch).ravel()
    sample = np.random.choice(range(vocab_size), p = predicted_probs)
        
    sequence_index.append(sample)

In [28]:
seq = ''.join(idx_to_char[c] for c in sequence_index)

In [29]:
seq

'9\nM:9/8\nL:1/8\nQ:1 1150|P"G\nP:Lig\nd|"G"G/2F/2G/2G/2 Bd/2B/2|"G"G/2A/2G/2F/2 ED/2G/2|\\\n"Am"A/2B/2c/2d/2 c/2B/2A/2G/2|"Am"FA A:|\nP:B\nA/2G/2|"D"F/2D/2D/2E/2 F/2D/2D/2E/2|"G"D/2E/2G/2F/2 GG/2A/2|"Em"B/2c/2B/2G/2 A/2B/2c/2d/2|\\\n"Em"e/2d/2c/2B/2 "Am"A/2B/2c/2d/2|"Am"eA A:|\nP:B ett A\nd/2c/2|B/2d/2=c/2d/2 e/2d/2c/2B/2|A/2g/2e/2f/2 g/2f/2e/2d/2|\\\nBA/2G/2 F/2D/2E/2G/2|\nF/2G/2A/2B/2 c/2B/2d/2B/2|c/2e/2f/2g/2 a/2e/2d/2c/2|\\\ng/2d/2c/2B/2 A/2G/2F/2E/2|G/2F/2E/2D/2 B/2E/2D/2E/2|\nG/4B/4A/2G/2 F/2G/2F/2E/2|G/2E/2E/2G/2 F/2G/2A/2B/2|\\\n [1A B/2A/2\nP:3\nB/2A/2||\nP:t. 4\nf/2 z/2z/2]z/2 |d/2z/2e/2 F/2D/2d/2c/2|B/2d/2e/2F/2 a/2A/2b/2e/2|\\\nK:D\na/4f/8g/8a/4c/4d/4f/4 g/4f/4f/4g/4f/4e/4|d/4B/4A/8B/4A/4A/4B/4 c/4d/4e/4f/4e/4d/4c/4B/4A/4G/4|\\\nK:D\nF/2f/2e/2d/2c d/2A/2c/2B/2|c/2d/2e/2f/2 d/2c/2d/2e/2|\\\nK:D\nf/4F/4G/4F/4E/4e/4f/4e/4 d/4F/4F/4D/4E/4F/4E/4A/4A/4G/4|F/4G/4B/4A/4A/4G/4A/4A/4 F/4G/4B/4A/4A/4G/4F/4E/4A/4G/4|\\\nD/4E/8C/4E/4C/4F/4G/4F/4D/4E/4 B/4A/4B/4A/4A/4G/4A/4B/4A/4G/4|\\\

In [30]:

for i in seq:
  if i=="\n":
    print(end="\n")
  else:
    print(i,end="")

9
M:9/8
L:1/8
Q:1 1150|P"G
P:Lig
d|"G"G/2F/2G/2G/2 Bd/2B/2|"G"G/2A/2G/2F/2 ED/2G/2|\
"Am"A/2B/2c/2d/2 c/2B/2A/2G/2|"Am"FA A:|
P:B
A/2G/2|"D"F/2D/2D/2E/2 F/2D/2D/2E/2|"G"D/2E/2G/2F/2 GG/2A/2|"Em"B/2c/2B/2G/2 A/2B/2c/2d/2|\
"Em"e/2d/2c/2B/2 "Am"A/2B/2c/2d/2|"Am"eA A:|
P:B ett A
d/2c/2|B/2d/2=c/2d/2 e/2d/2c/2B/2|A/2g/2e/2f/2 g/2f/2e/2d/2|\
BA/2G/2 F/2D/2E/2G/2|
F/2G/2A/2B/2 c/2B/2d/2B/2|c/2e/2f/2g/2 a/2e/2d/2c/2|\
g/2d/2c/2B/2 A/2G/2F/2E/2|G/2F/2E/2D/2 B/2E/2D/2E/2|
G/4B/4A/2G/2 F/2G/2F/2E/2|G/2E/2E/2G/2 F/2G/2A/2B/2|\
 [1A B/2A/2
P:3
B/2A/2||
P:t. 4
f/2 z/2z/2]z/2 |d/2z/2e/2 F/2D/2d/2c/2|B/2d/2e/2F/2 a/2A/2b/2e/2|\
K:D
a/4f/8g/8a/4c/4d/4f/4 g/4f/4f/4g/4f/4e/4|d/4B/4A/8B/4A/4A/4B/4 c/4d/4e/4f/4e/4d/4c/4B/4A/4G/4|\
K:D
F/2f/2e/2d/2c d/2A/2c/2B/2|c/2d/2e/2f/2 d/2c/2d/2e/2|\
K:D
f/4F/4G/4F/4E/4e/4f/4e/4 d/4F/4F/4D/4E/4F/4E/4A/4A/4G/4|F/4G/4B/4A/4A/4G/4A/4A/4 F/4G/4B/4A/4A/4G/4F/4E/4A/4G/4|\
D/4E/8C/4E/4C/4F/4G/4F/4D/4E/4 B/4A/4B/4A/4A/4G/4A/4B/4A/4G/4|\
K:D
F/2f/2e z/2d/2f|bF bc'||
K:D
F/2d/

In [31]:
# use this link (copy paste the AI generated music notes)
# https://www.abcjs.net/abcjs-editor.html